In [1]:
# https://www.kaggle.com/mathormad/inceptionv3-baseline-lb-0-379/code
# fork of scratch8, 29

In [2]:
%matplotlib inline
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle
import keras_metrics

import warnings
warnings.filterwarnings("ignore")
SIZE = 512

Using TensorFlow backend.


In [3]:
# https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric/notebook
import tensorflow as tf

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1) + (1 - K.mean(f1))

def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
        tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
        fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
        fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

        p = tp / (tp + fp + K.epsilon())
        r = tp / (tp + fn + K.epsilon())

        f1 = 2*p*r / (p+r+K.epsilon())
        f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1) +\
            (1-K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*f1, axis=-1))
    
    return weighted_loss

In [4]:
# Load dataset info
path_to_train = '../data/train/'
data = pd.read_csv('../data/train.csv')

In [5]:
data.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [6]:
train_dataset_info = []
for name, labels in zip(data['Id'], data['Target'].str.split(' ')):
    train_dataset_info.append({
        'path':os.path.join(path_to_train, name),
        'labels':np.array([int(label) for label in labels])})
train_dataset_info = np.array(train_dataset_info)


In [7]:
train_dataset_info

array([{'path': '../data/train/00070df0-bbc3-11e8-b2bc-ac1f6b6435d0', 'labels': array([16,  0])},
       {'path': '../data/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0', 'labels': array([7, 1, 2, 0])},
       {'path': '../data/train/000a9596-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([5])},
       ...,
       {'path': '../data/train/fff189d8-bbab-11e8-b2ba-ac1f6b6435d0', 'labels': array([7])},
       {'path': '../data/train/fffdf7e0-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([25,  2, 21])},
       {'path': '../data/train/fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0', 'labels': array([2, 0])}],
      dtype=object)

In [8]:
class data_generator:
    
    def create_train(dataset_info, batch_size, shape, augument=True):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(
                        X_train_batch[i]['path'], shape)
#                     image = tdi[i+start]
#                     image = cv2.resize(image, (shape[0], shape[1]))
                    if augument:
                        image = data_generator.augment(image)
                    batch_images.append(image/255.)
                    batch_labels[i][X_train_batch[i]['labels']] = 1
                yield np.array(batch_images, np.float32), batch_labels

    def load_image(path, shape):
        image_red_ch = Image.open(path+'_red.png')
        image_yellow_ch = Image.open(path+'_yellow.png')
        image_green_ch = Image.open(path+'_green.png')
        image_blue_ch = Image.open(path+'_blue.png')
        image1 = np.stack((
            np.array(image_red_ch),
            np.array(image_green_ch), 
            np.array(image_blue_ch)), -1)
        w, h = 512, 512
#         zero_data = np.zeros((h, w), dtype=np.uint8)
#         image2 = np.stack((
#             np.array(image_red_ch),
#             np.array(image_green_ch), 
#             np.array(image_yellow_ch)), -1)
#         image3 = np.stack((
#             np.array(image_yellow_ch),
#             np.array(image_green_ch), 
#             np.array(image_blue_ch)), -1)
# #         print(image1.shape, image2.shape)
#         image = np.vstack((image1, image2, image3))
#         print(image.shape)
        image =image1
#         image = canny_image4(image1)
        image = cv2.resize(image, (shape[0], shape[1]))
        return image
    
    def load_image2(path, shape):
        image_red_ch = Image.open(path+'_red.png')
        image_yellow_ch = Image.open(path+'_yellow.png')
        image_green_ch = Image.open(path+'_green.png')
        image_blue_ch = Image.open(path+'_blue.png')
        image1 = np.stack((
            np.array(image_red_ch),
            np.array(image_green_ch), 
            np.array(image_blue_ch)), -1)
        w, h = 512, 512
#         zero_data = np.zeros((h, w), dtype=np.uint8)
#         image2 = np.stack((
#             np.array(image_red_ch),
#             np.array(image_green_ch), 
#             np.array(image_yellow_ch)), -1)
#         image3 = np.stack((
#             np.array(image_yellow_ch),
#             np.array(image_green_ch), 
#             np.array(image_blue_ch)), -1)
# #         print(image1.shape, image2.shape)
#         image = np.vstack((image1, image2, image3))
#         print(image.shape)
        image =image1
#         image = canny_image4(image1)
        image = cv2.resize(image, (shape[0], shape[1]))
        return image
    
    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug
    def augment2(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                    iaa.Fliplr(0.5), # horizontal flips
                    iaa.Affine(rotate=0),
                    iaa.Affine(rotate=90),
                    iaa.Affine(rotate=180),
                    iaa.Affine(rotate=270),
                    iaa.Flipud(0.5),
                    iaa.Crop(percent=(0, 0.1)), # random crops
                    # Small gaussian blur with random sigma between 0 and 0.5.
                    # But we only blur about 50% of all images.
                    iaa.Sometimes(0.5,
                        iaa.GaussianBlur(sigma=(0, 0.5))
                    ),
                    # Strengthen or weaken the contrast in each image.
                    iaa.ContrastNormalization((0.75, 1.5)),
                    # Add gaussian noise.
                    # For 50% of all images, we sample the noise once per pixel.
                    # For the other 50% of all images, we sample the noise per pixel AND
                    # channel. This can change the color (not only brightness) of the
                    # pixels.
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                    # Make some images brighter and some darker.
                    # In 20% of all cases, we sample the multiplier once per channel,
                    # which can end up changing the color of the images.
                    iaa.Multiply((0.8, 1.2), per_channel=0.2),
                    # Apply affine transformations to each image.
                    # Scale/zoom them, translate/move them, rotate them and shear them.
                    iaa.Affine(
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                        rotate=(-180, 180),
                        shear=(-8, 8)
                    )
                ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug


In [9]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import Input, Conv2D, Reshape, UpSampling2D
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

In [10]:
# model = InceptionV3(include_top=False,
#                    weights='imagenet',
#                    input_shape=(512,512,3))
# #     bn = BatchNormalization()(input_tensor)
# model.summary()
# Layer (type)                    Output Shape         Param #     Connected to                     
# ==================================================================================================
# input_4 (InputLayer)            (None, 512, 512, 3)  0                                            
# __________________________________________________________________________________________________
# conv2d_408 (Conv2D)             (None, 255, 255, 32) 864         input_4[0][0]                    
# __________________________________________________________________________________________________
# batch_normalization_408 (BatchN (None, 255, 255, 32) 96          conv2d_408[0][0]                 
# __________________________________________________________________________________________________
# activation_407 (Activation)     (None, 255, 255, 32) 0           batch_normalization_408[0][0]    
# __________________________________________________________________________________________________
# conv2d_409 (Conv2D)             (None, 253, 253, 32) 9216        activation_407[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_409 (BatchN (None, 253, 253, 32) 96          conv2d_409[0][0]                 
# __________________________________________________________________________________________________
# activation_408 (Activation)     (None, 253, 253, 32) 0           batch_normalization_409[0][0]    
# __________________________________________________________________________________________________
# conv2d_410 (Conv2D)             (None, 253, 253, 64) 18432       activation_408[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_410 (BatchN (None, 253, 253, 64) 192         conv2d_410[0][0]                 
# __________________________________________________________________________________________________
# activation_409 (Activation)     (None, 253, 253, 64) 0           batch_normalization_410[0][0]    
# __________________________________________________________________________________________________
# max_pooling2d_9 (MaxPooling2D)  (None, 126, 126, 64) 0           activation_409[0][0]             
# __________________________________________________________________________________________________
# conv2d_411 (Conv2D)             (None, 126, 126, 80) 5120        max_pooling2d_9[0][0]            
# __________________________________________________________________________________________________
# batch_normalization_411 (BatchN (None, 126, 126, 80) 240         conv2d_411[0][0]                 
# __________________________________________________________________________________________________
# activation_410 (Activation)     (None, 126, 126, 80) 0           batch_normalization_411[0][0]    
# __________________________________________________________________________________________________
# conv2d_412 (Conv2D)             (None, 124, 124, 192 138240      activation_410[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_412 (BatchN (None, 124, 124, 192 576         conv2d_412[0][0]                 
# __________________________________________________________________________________________________
# activation_411 (Activation)     (None, 124, 124, 192 0           batch_normalization_412[0][0]    
# __________________________________________________________________________________________________
# max_pooling2d_10 (MaxPooling2D) (None, 61, 61, 192)  0           activation_411[0][0]             
# __________________________________________________________________________________________________
# conv2d_416 (Conv2D)             (None, 61, 61, 64)   12288       max_pooling2d_10[0][0]           
# __________________________________________________________________________________________________
# batch_normalization_416 (BatchN (None, 61, 61, 64)   192         conv2d_416[0][0]                 
# __________________________________________________________________________________________________
# activation_415 (Activation)     (None, 61, 61, 64)   0           batch_normalization_416[0][0]    
# __________________________________________________________________________________________________
# conv2d_414 (Conv2D)             (None, 61, 61, 48)   9216        max_pooling2d_10[0][0]           
# __________________________________________________________________________________________________
# conv2d_417 (Conv2D)             (None, 61, 61, 96)   55296       activation_415[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_414 (BatchN (None, 61, 61, 48)   144         conv2d_414[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_417 (BatchN (None, 61, 61, 96)   288         conv2d_417[0][0]                 
# __________________________________________________________________________________________________
# activation_413 (Activation)     (None, 61, 61, 48)   0           batch_normalization_414[0][0]    
# __________________________________________________________________________________________________
# activation_416 (Activation)     (None, 61, 61, 96)   0           batch_normalization_417[0][0]    
# __________________________________________________________________________________________________
# average_pooling2d_3 (AveragePoo (None, 61, 61, 192)  0           max_pooling2d_10[0][0]           
# __________________________________________________________________________________________________
# conv2d_413 (Conv2D)             (None, 61, 61, 64)   12288       max_pooling2d_10[0][0]           
# __________________________________________________________________________________________________
# conv2d_415 (Conv2D)             (None, 61, 61, 64)   76800       activation_413[0][0]             
# __________________________________________________________________________________________________
# conv2d_418 (Conv2D)             (None, 61, 61, 96)   82944       activation_416[0][0]             
# __________________________________________________________________________________________________
# conv2d_419 (Conv2D)             (None, 61, 61, 32)   6144        average_pooling2d_3[0][0]        
# __________________________________________________________________________________________________
# batch_normalization_413 (BatchN (None, 61, 61, 64)   192         conv2d_413[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_415 (BatchN (None, 61, 61, 64)   192         conv2d_415[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_418 (BatchN (None, 61, 61, 96)   288         conv2d_418[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_419 (BatchN (None, 61, 61, 32)   96          conv2d_419[0][0]                 
# __________________________________________________________________________________________________
# activation_412 (Activation)     (None, 61, 61, 64)   0           batch_normalization_413[0][0]    
# __________________________________________________________________________________________________
# activation_414 (Activation)     (None, 61, 61, 64)   0           batch_normalization_415[0][0]    
# __________________________________________________________________________________________________
# activation_417 (Activation)     (None, 61, 61, 96)   0           batch_normalization_418[0][0]    
# __________________________________________________________________________________________________
# activation_418 (Activation)     (None, 61, 61, 32)   0           batch_normalization_419[0][0]    
# __________________________________________________________________________________________________
# mixed0 (Concatenate)            (None, 61, 61, 256)  0           activation_412[0][0]             
#                                                                  activation_414[0][0]             
#                                                                  activation_417[0][0]             
#                                                                  activation_418[0][0]             
# __________________________________________________________________________________________________
# conv2d_423 (Conv2D)             (None, 61, 61, 64)   16384       mixed0[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_423 (BatchN (None, 61, 61, 64)   192         conv2d_423[0][0]                 
# __________________________________________________________________________________________________
# activation_422 (Activation)     (None, 61, 61, 64)   0           batch_normalization_423[0][0]    
# __________________________________________________________________________________________________
# conv2d_421 (Conv2D)             (None, 61, 61, 48)   12288       mixed0[0][0]                     
# __________________________________________________________________________________________________
# conv2d_424 (Conv2D)             (None, 61, 61, 96)   55296       activation_422[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_421 (BatchN (None, 61, 61, 48)   144         conv2d_421[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_424 (BatchN (None, 61, 61, 96)   288         conv2d_424[0][0]                 
# __________________________________________________________________________________________________
# activation_420 (Activation)     (None, 61, 61, 48)   0           batch_normalization_421[0][0]    
# __________________________________________________________________________________________________
# activation_423 (Activation)     (None, 61, 61, 96)   0           batch_normalization_424[0][0]    
# __________________________________________________________________________________________________
# average_pooling2d_4 (AveragePoo (None, 61, 61, 256)  0           mixed0[0][0]                     
# __________________________________________________________________________________________________
# conv2d_420 (Conv2D)             (None, 61, 61, 64)   16384       mixed0[0][0]                     
# __________________________________________________________________________________________________
# conv2d_422 (Conv2D)             (None, 61, 61, 64)   76800       activation_420[0][0]             
# __________________________________________________________________________________________________
# conv2d_425 (Conv2D)             (None, 61, 61, 96)   82944       activation_423[0][0]             
# __________________________________________________________________________________________________
# conv2d_426 (Conv2D)             (None, 61, 61, 64)   16384       average_pooling2d_4[0][0]        
# __________________________________________________________________________________________________
# batch_normalization_420 (BatchN (None, 61, 61, 64)   192         conv2d_420[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_422 (BatchN (None, 61, 61, 64)   192         conv2d_422[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_425 (BatchN (None, 61, 61, 96)   288         conv2d_425[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_426 (BatchN (None, 61, 61, 64)   192         conv2d_426[0][0]                 
# __________________________________________________________________________________________________
# activation_419 (Activation)     (None, 61, 61, 64)   0           batch_normalization_420[0][0]    
# __________________________________________________________________________________________________
# activation_421 (Activation)     (None, 61, 61, 64)   0           batch_normalization_422[0][0]    
# __________________________________________________________________________________________________
# activation_424 (Activation)     (None, 61, 61, 96)   0           batch_normalization_425[0][0]    
# __________________________________________________________________________________________________
# activation_425 (Activation)     (None, 61, 61, 64)   0           batch_normalization_426[0][0]    
# __________________________________________________________________________________________________
# mixed1 (Concatenate)            (None, 61, 61, 288)  0           activation_419[0][0]             
#                                                                  activation_421[0][0]             
#                                                                  activation_424[0][0]             
#                                                                  activation_425[0][0]             
# __________________________________________________________________________________________________
# conv2d_430 (Conv2D)             (None, 61, 61, 64)   18432       mixed1[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_430 (BatchN (None, 61, 61, 64)   192         conv2d_430[0][0]                 
# __________________________________________________________________________________________________
# activation_429 (Activation)     (None, 61, 61, 64)   0           batch_normalization_430[0][0]    
# __________________________________________________________________________________________________
# conv2d_428 (Conv2D)             (None, 61, 61, 48)   13824       mixed1[0][0]                     
# __________________________________________________________________________________________________
# conv2d_431 (Conv2D)             (None, 61, 61, 96)   55296       activation_429[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_428 (BatchN (None, 61, 61, 48)   144         conv2d_428[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_431 (BatchN (None, 61, 61, 96)   288         conv2d_431[0][0]                 
# __________________________________________________________________________________________________
# activation_427 (Activation)     (None, 61, 61, 48)   0           batch_normalization_428[0][0]    
# __________________________________________________________________________________________________
# activation_430 (Activation)     (None, 61, 61, 96)   0           batch_normalization_431[0][0]    
# __________________________________________________________________________________________________
# average_pooling2d_5 (AveragePoo (None, 61, 61, 288)  0           mixed1[0][0]                     
# __________________________________________________________________________________________________
# conv2d_427 (Conv2D)             (None, 61, 61, 64)   18432       mixed1[0][0]                     
# __________________________________________________________________________________________________
# conv2d_429 (Conv2D)             (None, 61, 61, 64)   76800       activation_427[0][0]             
# __________________________________________________________________________________________________
# conv2d_432 (Conv2D)             (None, 61, 61, 96)   82944       activation_430[0][0]             
# __________________________________________________________________________________________________
# conv2d_433 (Conv2D)             (None, 61, 61, 64)   18432       average_pooling2d_5[0][0]        
# __________________________________________________________________________________________________
# batch_normalization_427 (BatchN (None, 61, 61, 64)   192         conv2d_427[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_429 (BatchN (None, 61, 61, 64)   192         conv2d_429[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_432 (BatchN (None, 61, 61, 96)   288         conv2d_432[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_433 (BatchN (None, 61, 61, 64)   192         conv2d_433[0][0]                 
# __________________________________________________________________________________________________
# activation_426 (Activation)     (None, 61, 61, 64)   0           batch_normalization_427[0][0]    
# __________________________________________________________________________________________________
# activation_428 (Activation)     (None, 61, 61, 64)   0           batch_normalization_429[0][0]    
# __________________________________________________________________________________________________
# activation_431 (Activation)     (None, 61, 61, 96)   0           batch_normalization_432[0][0]    
# __________________________________________________________________________________________________
# activation_432 (Activation)     (None, 61, 61, 64)   0           batch_normalization_433[0][0]    
# __________________________________________________________________________________________________
# mixed2 (Concatenate)            (None, 61, 61, 288)  0           activation_426[0][0]             
#                                                                  activation_428[0][0]             
#                                                                  activation_431[0][0]             
#                                                                  activation_432[0][0]             
# __________________________________________________________________________________________________
# conv2d_435 (Conv2D)             (None, 61, 61, 64)   18432       mixed2[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_435 (BatchN (None, 61, 61, 64)   192         conv2d_435[0][0]                 
# __________________________________________________________________________________________________
# activation_434 (Activation)     (None, 61, 61, 64)   0           batch_normalization_435[0][0]    
# __________________________________________________________________________________________________
# conv2d_436 (Conv2D)             (None, 61, 61, 96)   55296       activation_434[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_436 (BatchN (None, 61, 61, 96)   288         conv2d_436[0][0]                 
# __________________________________________________________________________________________________
# activation_435 (Activation)     (None, 61, 61, 96)   0           batch_normalization_436[0][0]    
# __________________________________________________________________________________________________
# conv2d_434 (Conv2D)             (None, 30, 30, 384)  995328      mixed2[0][0]                     
# __________________________________________________________________________________________________
# conv2d_437 (Conv2D)             (None, 30, 30, 96)   82944       activation_435[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_434 (BatchN (None, 30, 30, 384)  1152        conv2d_434[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_437 (BatchN (None, 30, 30, 96)   288         conv2d_437[0][0]                 
# __________________________________________________________________________________________________
# activation_433 (Activation)     (None, 30, 30, 384)  0           batch_normalization_434[0][0]    
# __________________________________________________________________________________________________
# activation_436 (Activation)     (None, 30, 30, 96)   0           batch_normalization_437[0][0]    
# __________________________________________________________________________________________________
# max_pooling2d_11 (MaxPooling2D) (None, 30, 30, 288)  0           mixed2[0][0]                     
# __________________________________________________________________________________________________
# mixed3 (Concatenate)            (None, 30, 30, 768)  0           activation_433[0][0]             
#                                                                  activation_436[0][0]             
#                                                                  max_pooling2d_11[0][0]           
# __________________________________________________________________________________________________
# conv2d_442 (Conv2D)             (None, 30, 30, 128)  98304       mixed3[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_442 (BatchN (None, 30, 30, 128)  384         conv2d_442[0][0]                 
# __________________________________________________________________________________________________
# activation_441 (Activation)     (None, 30, 30, 128)  0           batch_normalization_442[0][0]    
# __________________________________________________________________________________________________
# conv2d_443 (Conv2D)             (None, 30, 30, 128)  114688      activation_441[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_443 (BatchN (None, 30, 30, 128)  384         conv2d_443[0][0]                 
# __________________________________________________________________________________________________
# activation_442 (Activation)     (None, 30, 30, 128)  0           batch_normalization_443[0][0]    
# __________________________________________________________________________________________________
# conv2d_439 (Conv2D)             (None, 30, 30, 128)  98304       mixed3[0][0]                     
# __________________________________________________________________________________________________
# conv2d_444 (Conv2D)             (None, 30, 30, 128)  114688      activation_442[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_439 (BatchN (None, 30, 30, 128)  384         conv2d_439[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_444 (BatchN (None, 30, 30, 128)  384         conv2d_444[0][0]                 
# __________________________________________________________________________________________________
# activation_438 (Activation)     (None, 30, 30, 128)  0           batch_normalization_439[0][0]    
# __________________________________________________________________________________________________
# activation_443 (Activation)     (None, 30, 30, 128)  0           batch_normalization_444[0][0]    
# __________________________________________________________________________________________________
# conv2d_440 (Conv2D)             (None, 30, 30, 128)  114688      activation_438[0][0]             
# __________________________________________________________________________________________________
# conv2d_445 (Conv2D)             (None, 30, 30, 128)  114688      activation_443[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_440 (BatchN (None, 30, 30, 128)  384         conv2d_440[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_445 (BatchN (None, 30, 30, 128)  384         conv2d_445[0][0]                 
# __________________________________________________________________________________________________
# activation_439 (Activation)     (None, 30, 30, 128)  0           batch_normalization_440[0][0]    
# __________________________________________________________________________________________________
# activation_444 (Activation)     (None, 30, 30, 128)  0           batch_normalization_445[0][0]    
# __________________________________________________________________________________________________
# average_pooling2d_6 (AveragePoo (None, 30, 30, 768)  0           mixed3[0][0]                     
# __________________________________________________________________________________________________
# conv2d_438 (Conv2D)             (None, 30, 30, 192)  147456      mixed3[0][0]                     
# __________________________________________________________________________________________________
# conv2d_441 (Conv2D)             (None, 30, 30, 192)  172032      activation_439[0][0]             
# __________________________________________________________________________________________________
# conv2d_446 (Conv2D)             (None, 30, 30, 192)  172032      activation_444[0][0]             
# __________________________________________________________________________________________________
# conv2d_447 (Conv2D)             (None, 30, 30, 192)  147456      average_pooling2d_6[0][0]        
# __________________________________________________________________________________________________
# batch_normalization_438 (BatchN (None, 30, 30, 192)  576         conv2d_438[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_441 (BatchN (None, 30, 30, 192)  576         conv2d_441[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_446 (BatchN (None, 30, 30, 192)  576         conv2d_446[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_447 (BatchN (None, 30, 30, 192)  576         conv2d_447[0][0]                 
# __________________________________________________________________________________________________
# activation_437 (Activation)     (None, 30, 30, 192)  0           batch_normalization_438[0][0]    
# __________________________________________________________________________________________________
# activation_440 (Activation)     (None, 30, 30, 192)  0           batch_normalization_441[0][0]    
# __________________________________________________________________________________________________
# activation_445 (Activation)     (None, 30, 30, 192)  0           batch_normalization_446[0][0]    
# __________________________________________________________________________________________________
# activation_446 (Activation)     (None, 30, 30, 192)  0           batch_normalization_447[0][0]    
# __________________________________________________________________________________________________
# mixed4 (Concatenate)            (None, 30, 30, 768)  0           activation_437[0][0]             
#                                                                  activation_440[0][0]             
#                                                                  activation_445[0][0]             
#                                                                  activation_446[0][0]             
# __________________________________________________________________________________________________
# conv2d_452 (Conv2D)             (None, 30, 30, 160)  122880      mixed4[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_452 (BatchN (None, 30, 30, 160)  480         conv2d_452[0][0]                 
# __________________________________________________________________________________________________
# activation_451 (Activation)     (None, 30, 30, 160)  0           batch_normalization_452[0][0]    
# __________________________________________________________________________________________________
# conv2d_453 (Conv2D)             (None, 30, 30, 160)  179200      activation_451[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_453 (BatchN (None, 30, 30, 160)  480         conv2d_453[0][0]                 
# __________________________________________________________________________________________________
# activation_452 (Activation)     (None, 30, 30, 160)  0           batch_normalization_453[0][0]    
# __________________________________________________________________________________________________
# conv2d_449 (Conv2D)             (None, 30, 30, 160)  122880      mixed4[0][0]                     
# __________________________________________________________________________________________________
# conv2d_454 (Conv2D)             (None, 30, 30, 160)  179200      activation_452[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_449 (BatchN (None, 30, 30, 160)  480         conv2d_449[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_454 (BatchN (None, 30, 30, 160)  480         conv2d_454[0][0]                 
# __________________________________________________________________________________________________
# activation_448 (Activation)     (None, 30, 30, 160)  0           batch_normalization_449[0][0]    
# __________________________________________________________________________________________________
# activation_453 (Activation)     (None, 30, 30, 160)  0           batch_normalization_454[0][0]    
# __________________________________________________________________________________________________
# conv2d_450 (Conv2D)             (None, 30, 30, 160)  179200      activation_448[0][0]             
# __________________________________________________________________________________________________
# conv2d_455 (Conv2D)             (None, 30, 30, 160)  179200      activation_453[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_450 (BatchN (None, 30, 30, 160)  480         conv2d_450[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_455 (BatchN (None, 30, 30, 160)  480         conv2d_455[0][0]                 
# __________________________________________________________________________________________________
# activation_449 (Activation)     (None, 30, 30, 160)  0           batch_normalization_450[0][0]    
# __________________________________________________________________________________________________
# activation_454 (Activation)     (None, 30, 30, 160)  0           batch_normalization_455[0][0]    
# __________________________________________________________________________________________________
# average_pooling2d_7 (AveragePoo (None, 30, 30, 768)  0           mixed4[0][0]                     
# __________________________________________________________________________________________________
# conv2d_448 (Conv2D)             (None, 30, 30, 192)  147456      mixed4[0][0]                     
# __________________________________________________________________________________________________
# conv2d_451 (Conv2D)             (None, 30, 30, 192)  215040      activation_449[0][0]             
# __________________________________________________________________________________________________
# conv2d_456 (Conv2D)             (None, 30, 30, 192)  215040      activation_454[0][0]             
# __________________________________________________________________________________________________
# conv2d_457 (Conv2D)             (None, 30, 30, 192)  147456      average_pooling2d_7[0][0]        
# __________________________________________________________________________________________________
# batch_normalization_448 (BatchN (None, 30, 30, 192)  576         conv2d_448[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_451 (BatchN (None, 30, 30, 192)  576         conv2d_451[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_456 (BatchN (None, 30, 30, 192)  576         conv2d_456[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_457 (BatchN (None, 30, 30, 192)  576         conv2d_457[0][0]                 
# __________________________________________________________________________________________________
# activation_447 (Activation)     (None, 30, 30, 192)  0           batch_normalization_448[0][0]    
# __________________________________________________________________________________________________
# activation_450 (Activation)     (None, 30, 30, 192)  0           batch_normalization_451[0][0]    
# __________________________________________________________________________________________________
# activation_455 (Activation)     (None, 30, 30, 192)  0           batch_normalization_456[0][0]    
# __________________________________________________________________________________________________
# activation_456 (Activation)     (None, 30, 30, 192)  0           batch_normalization_457[0][0]    
# __________________________________________________________________________________________________
# mixed5 (Concatenate)            (None, 30, 30, 768)  0           activation_447[0][0]             
#                                                                  activation_450[0][0]             
#                                                                  activation_455[0][0]             
#                                                                  activation_456[0][0]             
# __________________________________________________________________________________________________
# conv2d_462 (Conv2D)             (None, 30, 30, 160)  122880      mixed5[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_462 (BatchN (None, 30, 30, 160)  480         conv2d_462[0][0]                 
# __________________________________________________________________________________________________
# activation_461 (Activation)     (None, 30, 30, 160)  0           batch_normalization_462[0][0]    
# __________________________________________________________________________________________________
# conv2d_463 (Conv2D)             (None, 30, 30, 160)  179200      activation_461[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_463 (BatchN (None, 30, 30, 160)  480         conv2d_463[0][0]                 
# __________________________________________________________________________________________________
# activation_462 (Activation)     (None, 30, 30, 160)  0           batch_normalization_463[0][0]    
# __________________________________________________________________________________________________
# conv2d_459 (Conv2D)             (None, 30, 30, 160)  122880      mixed5[0][0]                     
# __________________________________________________________________________________________________
# conv2d_464 (Conv2D)             (None, 30, 30, 160)  179200      activation_462[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_459 (BatchN (None, 30, 30, 160)  480         conv2d_459[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_464 (BatchN (None, 30, 30, 160)  480         conv2d_464[0][0]                 
# __________________________________________________________________________________________________
# activation_458 (Activation)     (None, 30, 30, 160)  0           batch_normalization_459[0][0]    
# __________________________________________________________________________________________________
# activation_463 (Activation)     (None, 30, 30, 160)  0           batch_normalization_464[0][0]    
# __________________________________________________________________________________________________
# conv2d_460 (Conv2D)             (None, 30, 30, 160)  179200      activation_458[0][0]             
# __________________________________________________________________________________________________
# conv2d_465 (Conv2D)             (None, 30, 30, 160)  179200      activation_463[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_460 (BatchN (None, 30, 30, 160)  480         conv2d_460[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_465 (BatchN (None, 30, 30, 160)  480         conv2d_465[0][0]                 
# __________________________________________________________________________________________________
# activation_459 (Activation)     (None, 30, 30, 160)  0           batch_normalization_460[0][0]    
# __________________________________________________________________________________________________
# activation_464 (Activation)     (None, 30, 30, 160)  0           batch_normalization_465[0][0]    
# __________________________________________________________________________________________________
# average_pooling2d_8 (AveragePoo (None, 30, 30, 768)  0           mixed5[0][0]                     
# __________________________________________________________________________________________________
# conv2d_458 (Conv2D)             (None, 30, 30, 192)  147456      mixed5[0][0]                     
# __________________________________________________________________________________________________
# conv2d_461 (Conv2D)             (None, 30, 30, 192)  215040      activation_459[0][0]             
# __________________________________________________________________________________________________
# conv2d_466 (Conv2D)             (None, 30, 30, 192)  215040      activation_464[0][0]             
# __________________________________________________________________________________________________
# conv2d_467 (Conv2D)             (None, 30, 30, 192)  147456      average_pooling2d_8[0][0]        
# __________________________________________________________________________________________________
# batch_normalization_458 (BatchN (None, 30, 30, 192)  576         conv2d_458[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_461 (BatchN (None, 30, 30, 192)  576         conv2d_461[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_466 (BatchN (None, 30, 30, 192)  576         conv2d_466[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_467 (BatchN (None, 30, 30, 192)  576         conv2d_467[0][0]                 
# __________________________________________________________________________________________________
# activation_457 (Activation)     (None, 30, 30, 192)  0           batch_normalization_458[0][0]    
# __________________________________________________________________________________________________
# activation_460 (Activation)     (None, 30, 30, 192)  0           batch_normalization_461[0][0]    
# __________________________________________________________________________________________________
# activation_465 (Activation)     (None, 30, 30, 192)  0           batch_normalization_466[0][0]    
# __________________________________________________________________________________________________
# activation_466 (Activation)     (None, 30, 30, 192)  0           batch_normalization_467[0][0]    
# __________________________________________________________________________________________________
# mixed6 (Concatenate)            (None, 30, 30, 768)  0           activation_457[0][0]             
#                                                                  activation_460[0][0]             
#                                                                  activation_465[0][0]             
#                                                                  activation_466[0][0]             
# __________________________________________________________________________________________________
# conv2d_472 (Conv2D)             (None, 30, 30, 192)  147456      mixed6[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_472 (BatchN (None, 30, 30, 192)  576         conv2d_472[0][0]                 
# __________________________________________________________________________________________________
# activation_471 (Activation)     (None, 30, 30, 192)  0           batch_normalization_472[0][0]    
# __________________________________________________________________________________________________
# conv2d_473 (Conv2D)             (None, 30, 30, 192)  258048      activation_471[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_473 (BatchN (None, 30, 30, 192)  576         conv2d_473[0][0]                 
# __________________________________________________________________________________________________
# activation_472 (Activation)     (None, 30, 30, 192)  0           batch_normalization_473[0][0]    
# __________________________________________________________________________________________________
# conv2d_469 (Conv2D)             (None, 30, 30, 192)  147456      mixed6[0][0]                     
# __________________________________________________________________________________________________
# conv2d_474 (Conv2D)             (None, 30, 30, 192)  258048      activation_472[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_469 (BatchN (None, 30, 30, 192)  576         conv2d_469[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_474 (BatchN (None, 30, 30, 192)  576         conv2d_474[0][0]                 
# __________________________________________________________________________________________________
# activation_468 (Activation)     (None, 30, 30, 192)  0           batch_normalization_469[0][0]    
# __________________________________________________________________________________________________
# activation_473 (Activation)     (None, 30, 30, 192)  0           batch_normalization_474[0][0]    
# __________________________________________________________________________________________________
# conv2d_470 (Conv2D)             (None, 30, 30, 192)  258048      activation_468[0][0]             
# __________________________________________________________________________________________________
# conv2d_475 (Conv2D)             (None, 30, 30, 192)  258048      activation_473[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_470 (BatchN (None, 30, 30, 192)  576         conv2d_470[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_475 (BatchN (None, 30, 30, 192)  576         conv2d_475[0][0]                 
# __________________________________________________________________________________________________
# activation_469 (Activation)     (None, 30, 30, 192)  0           batch_normalization_470[0][0]    
# __________________________________________________________________________________________________
# activation_474 (Activation)     (None, 30, 30, 192)  0           batch_normalization_475[0][0]    
# __________________________________________________________________________________________________
# average_pooling2d_9 (AveragePoo (None, 30, 30, 768)  0           mixed6[0][0]                     
# __________________________________________________________________________________________________
# conv2d_468 (Conv2D)             (None, 30, 30, 192)  147456      mixed6[0][0]                     
# __________________________________________________________________________________________________
# conv2d_471 (Conv2D)             (None, 30, 30, 192)  258048      activation_469[0][0]             
# __________________________________________________________________________________________________
# conv2d_476 (Conv2D)             (None, 30, 30, 192)  258048      activation_474[0][0]             
# __________________________________________________________________________________________________
# conv2d_477 (Conv2D)             (None, 30, 30, 192)  147456      average_pooling2d_9[0][0]        
# __________________________________________________________________________________________________
# batch_normalization_468 (BatchN (None, 30, 30, 192)  576         conv2d_468[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_471 (BatchN (None, 30, 30, 192)  576         conv2d_471[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_476 (BatchN (None, 30, 30, 192)  576         conv2d_476[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_477 (BatchN (None, 30, 30, 192)  576         conv2d_477[0][0]                 
# __________________________________________________________________________________________________
# activation_467 (Activation)     (None, 30, 30, 192)  0           batch_normalization_468[0][0]    
# __________________________________________________________________________________________________
# activation_470 (Activation)     (None, 30, 30, 192)  0           batch_normalization_471[0][0]    
# __________________________________________________________________________________________________
# activation_475 (Activation)     (None, 30, 30, 192)  0           batch_normalization_476[0][0]    
# __________________________________________________________________________________________________
# activation_476 (Activation)     (None, 30, 30, 192)  0           batch_normalization_477[0][0]    
# __________________________________________________________________________________________________
# mixed7 (Concatenate)            (None, 30, 30, 768)  0           activation_467[0][0]             
#                                                                  activation_470[0][0]             
#                                                                  activation_475[0][0]             
#                                                                  activation_476[0][0]             
# __________________________________________________________________________________________________
# conv2d_480 (Conv2D)             (None, 30, 30, 192)  147456      mixed7[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_480 (BatchN (None, 30, 30, 192)  576         conv2d_480[0][0]                 
# __________________________________________________________________________________________________
# activation_479 (Activation)     (None, 30, 30, 192)  0           batch_normalization_480[0][0]    
# __________________________________________________________________________________________________
# conv2d_481 (Conv2D)             (None, 30, 30, 192)  258048      activation_479[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_481 (BatchN (None, 30, 30, 192)  576         conv2d_481[0][0]                 
# __________________________________________________________________________________________________
# activation_480 (Activation)     (None, 30, 30, 192)  0           batch_normalization_481[0][0]    
# __________________________________________________________________________________________________
# conv2d_478 (Conv2D)             (None, 30, 30, 192)  147456      mixed7[0][0]                     
# __________________________________________________________________________________________________
# conv2d_482 (Conv2D)             (None, 30, 30, 192)  258048      activation_480[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_478 (BatchN (None, 30, 30, 192)  576         conv2d_478[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_482 (BatchN (None, 30, 30, 192)  576         conv2d_482[0][0]                 
# __________________________________________________________________________________________________
# activation_477 (Activation)     (None, 30, 30, 192)  0           batch_normalization_478[0][0]    
# __________________________________________________________________________________________________
# activation_481 (Activation)     (None, 30, 30, 192)  0           batch_normalization_482[0][0]    
# __________________________________________________________________________________________________
# conv2d_479 (Conv2D)             (None, 14, 14, 320)  552960      activation_477[0][0]             
# __________________________________________________________________________________________________
# conv2d_483 (Conv2D)             (None, 14, 14, 192)  331776      activation_481[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_479 (BatchN (None, 14, 14, 320)  960         conv2d_479[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_483 (BatchN (None, 14, 14, 192)  576         conv2d_483[0][0]                 
# __________________________________________________________________________________________________
# activation_478 (Activation)     (None, 14, 14, 320)  0           batch_normalization_479[0][0]    
# __________________________________________________________________________________________________
# activation_482 (Activation)     (None, 14, 14, 192)  0           batch_normalization_483[0][0]    
# __________________________________________________________________________________________________
# max_pooling2d_12 (MaxPooling2D) (None, 14, 14, 768)  0           mixed7[0][0]                     
# __________________________________________________________________________________________________
# mixed8 (Concatenate)            (None, 14, 14, 1280) 0           activation_478[0][0]             
#                                                                  activation_482[0][0]             
#                                                                  max_pooling2d_12[0][0]           
# __________________________________________________________________________________________________
# conv2d_488 (Conv2D)             (None, 14, 14, 448)  573440      mixed8[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_488 (BatchN (None, 14, 14, 448)  1344        conv2d_488[0][0]                 
# __________________________________________________________________________________________________
# activation_487 (Activation)     (None, 14, 14, 448)  0           batch_normalization_488[0][0]    
# __________________________________________________________________________________________________
# conv2d_485 (Conv2D)             (None, 14, 14, 384)  491520      mixed8[0][0]                     
# __________________________________________________________________________________________________
# conv2d_489 (Conv2D)             (None, 14, 14, 384)  1548288     activation_487[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_485 (BatchN (None, 14, 14, 384)  1152        conv2d_485[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_489 (BatchN (None, 14, 14, 384)  1152        conv2d_489[0][0]                 
# __________________________________________________________________________________________________
# activation_484 (Activation)     (None, 14, 14, 384)  0           batch_normalization_485[0][0]    
# __________________________________________________________________________________________________
# activation_488 (Activation)     (None, 14, 14, 384)  0           batch_normalization_489[0][0]    
# __________________________________________________________________________________________________
# conv2d_486 (Conv2D)             (None, 14, 14, 384)  442368      activation_484[0][0]             
# __________________________________________________________________________________________________
# conv2d_487 (Conv2D)             (None, 14, 14, 384)  442368      activation_484[0][0]             
# __________________________________________________________________________________________________
# conv2d_490 (Conv2D)             (None, 14, 14, 384)  442368      activation_488[0][0]             
# __________________________________________________________________________________________________
# conv2d_491 (Conv2D)             (None, 14, 14, 384)  442368      activation_488[0][0]             
# __________________________________________________________________________________________________
# average_pooling2d_10 (AveragePo (None, 14, 14, 1280) 0           mixed8[0][0]                     
# __________________________________________________________________________________________________
# conv2d_484 (Conv2D)             (None, 14, 14, 320)  409600      mixed8[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_486 (BatchN (None, 14, 14, 384)  1152        conv2d_486[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_487 (BatchN (None, 14, 14, 384)  1152        conv2d_487[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_490 (BatchN (None, 14, 14, 384)  1152        conv2d_490[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_491 (BatchN (None, 14, 14, 384)  1152        conv2d_491[0][0]                 
# __________________________________________________________________________________________________
# conv2d_492 (Conv2D)             (None, 14, 14, 192)  245760      average_pooling2d_10[0][0]       
# __________________________________________________________________________________________________
# batch_normalization_484 (BatchN (None, 14, 14, 320)  960         conv2d_484[0][0]                 
# __________________________________________________________________________________________________
# activation_485 (Activation)     (None, 14, 14, 384)  0           batch_normalization_486[0][0]    
# __________________________________________________________________________________________________
# activation_486 (Activation)     (None, 14, 14, 384)  0           batch_normalization_487[0][0]    
# __________________________________________________________________________________________________
# activation_489 (Activation)     (None, 14, 14, 384)  0           batch_normalization_490[0][0]    
# __________________________________________________________________________________________________
# activation_490 (Activation)     (None, 14, 14, 384)  0           batch_normalization_491[0][0]    
# __________________________________________________________________________________________________
# batch_normalization_492 (BatchN (None, 14, 14, 192)  576         conv2d_492[0][0]                 
# __________________________________________________________________________________________________
# activation_483 (Activation)     (None, 14, 14, 320)  0           batch_normalization_484[0][0]    
# __________________________________________________________________________________________________
# mixed9_0 (Concatenate)          (None, 14, 14, 768)  0           activation_485[0][0]             
#                                                                  activation_486[0][0]             
# __________________________________________________________________________________________________
# concatenate_1 (Concatenate)     (None, 14, 14, 768)  0           activation_489[0][0]             
#                                                                  activation_490[0][0]             
# __________________________________________________________________________________________________
# activation_491 (Activation)     (None, 14, 14, 192)  0           batch_normalization_492[0][0]    
# __________________________________________________________________________________________________
# mixed9 (Concatenate)            (None, 14, 14, 2048) 0           activation_483[0][0]             
#                                                                  mixed9_0[0][0]                   
#                                                                  concatenate_1[0][0]              
#                                                                  activation_491[0][0]             
# __________________________________________________________________________________________________
# conv2d_497 (Conv2D)             (None, 14, 14, 448)  917504      mixed9[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_497 (BatchN (None, 14, 14, 448)  1344        conv2d_497[0][0]                 
# __________________________________________________________________________________________________
# activation_496 (Activation)     (None, 14, 14, 448)  0           batch_normalization_497[0][0]    
# __________________________________________________________________________________________________
# conv2d_494 (Conv2D)             (None, 14, 14, 384)  786432      mixed9[0][0]                     
# __________________________________________________________________________________________________
# conv2d_498 (Conv2D)             (None, 14, 14, 384)  1548288     activation_496[0][0]             
# __________________________________________________________________________________________________
# batch_normalization_494 (BatchN (None, 14, 14, 384)  1152        conv2d_494[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_498 (BatchN (None, 14, 14, 384)  1152        conv2d_498[0][0]                 
# __________________________________________________________________________________________________
# activation_493 (Activation)     (None, 14, 14, 384)  0           batch_normalization_494[0][0]    
# __________________________________________________________________________________________________
# activation_497 (Activation)     (None, 14, 14, 384)  0           batch_normalization_498[0][0]    
# __________________________________________________________________________________________________
# conv2d_495 (Conv2D)             (None, 14, 14, 384)  442368      activation_493[0][0]             
# __________________________________________________________________________________________________
# conv2d_496 (Conv2D)             (None, 14, 14, 384)  442368      activation_493[0][0]             
# __________________________________________________________________________________________________
# conv2d_499 (Conv2D)             (None, 14, 14, 384)  442368      activation_497[0][0]             
# __________________________________________________________________________________________________
# conv2d_500 (Conv2D)             (None, 14, 14, 384)  442368      activation_497[0][0]             
# __________________________________________________________________________________________________
# average_pooling2d_11 (AveragePo (None, 14, 14, 2048) 0           mixed9[0][0]                     
# __________________________________________________________________________________________________
# conv2d_493 (Conv2D)             (None, 14, 14, 320)  655360      mixed9[0][0]                     
# __________________________________________________________________________________________________
# batch_normalization_495 (BatchN (None, 14, 14, 384)  1152        conv2d_495[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_496 (BatchN (None, 14, 14, 384)  1152        conv2d_496[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_499 (BatchN (None, 14, 14, 384)  1152        conv2d_499[0][0]                 
# __________________________________________________________________________________________________
# batch_normalization_500 (BatchN (None, 14, 14, 384)  1152        conv2d_500[0][0]                 
# __________________________________________________________________________________________________
# conv2d_501 (Conv2D)             (None, 14, 14, 192)  393216      average_pooling2d_11[0][0]       
# __________________________________________________________________________________________________
# batch_normalization_493 (BatchN (None, 14, 14, 320)  960         conv2d_493[0][0]                 
# __________________________________________________________________________________________________
# activation_494 (Activation)     (None, 14, 14, 384)  0           batch_normalization_495[0][0]    
# __________________________________________________________________________________________________
# activation_495 (Activation)     (None, 14, 14, 384)  0           batch_normalization_496[0][0]    
# __________________________________________________________________________________________________
# activation_498 (Activation)     (None, 14, 14, 384)  0           batch_normalization_499[0][0]    
# __________________________________________________________________________________________________
# activation_499 (Activation)     (None, 14, 14, 384)  0           batch_normalization_500[0][0]    
# __________________________________________________________________________________________________
# batch_normalization_501 (BatchN (None, 14, 14, 192)  576         conv2d_501[0][0]                 
# __________________________________________________________________________________________________
# activation_492 (Activation)     (None, 14, 14, 320)  0           batch_normalization_493[0][0]    
# __________________________________________________________________________________________________
# mixed9_1 (Concatenate)          (None, 14, 14, 768)  0           activation_494[0][0]             
#                                                                  activation_495[0][0]             
# __________________________________________________________________________________________________
# concatenate_2 (Concatenate)     (None, 14, 14, 768)  0           activation_498[0][0]             
#                                                                  activation_499[0][0]             
# __________________________________________________________________________________________________
# activation_500 (Activation)     (None, 14, 14, 192)  0           batch_normalization_501[0][0]    
# __________________________________________________________________________________________________
# mixed10 (Concatenate)           (None, 14, 14, 2048) 0           activation_492[0][0]             
#                                                                  mixed9_1[0][0]                   
#                                                                  concatenate_2[0][0]              
#                                                                  activation_500[0][0]             
# ==================================================================================================
# Total params: 21,802,784
# Trainable params: 21,768,352
# Non-trainable params: 34,432
# __________________________________________________________________________________________________

# # model = InceptionResNetV2(include_top=False,
# #                    weights='imagenet',
# #                    input_shape=(512,512,3))
# # #     bn = BatchNormalization()(input_tensor)
# # model.summary()

In [11]:
def decoder_block(x,blocks=5,start_filters=256):
    for i in range(blocks):
        x = Conv2D(start_filters//(2**i),(3,3), activation='relu', padding='same')(x)
        x = Conv2D(start_filters//(2**i),(3,3), activation='relu', padding='same')(x)
        x = UpSampling2D()(x)
    
    return x

In [12]:
def create_model2(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape)
    bn = BatchNormalization(name='bn1')(input_tensor)
    x = base_model(bn)
    x1 = x
    x = GlobalAveragePooling2D()(x)
#     print(x.shape)
#     x = Conv2D(32, kernel_size=(1,1), activation='relu', name='conv1')(x)
#     y = Dense(14 * 14 * 2048)(x)
#     x = Reshape((shape[1], shape[2], shape[3]))(x)
#     y = Reshape((16, 16, 1568))(x1)
#     x = Dropout(0.5)(x)
#     y = Reshape(16,16,1568)(y)
#     y = decoder_block(y)
#     y = Dense(3, activation='relu')(y)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model

In [ ]:
def create_model(input_shape1, input_shape2, input_shape3, input_shape4, n_out):
    input_tensor1 = Input(shape=input_shape1)
    
    input_tensor2 = Input(shape=input_shape2)
    
    input_tensor3 = Input(shape=input_shape3)
    
    input_tensor4 = Input(shape=input_shape4)
    
    x1 = Conv2D(64, kernel_size=(5,5), activation='relu', name='conv1a')(input_tensor1)
    x1 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x1)
    
    x1 = Conv2D(128, kernel_size=(3,3), activation='relu', name='conv2a')(x1)
    x1 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x1)
    
    x1 = Conv2D(256, kernel_size=(3,3), activation='relu', name='conv3a')(x1)
    x1 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x1)
    
    x2 = Conv2D(64, kernel_size=(5,5), activation='relu', name='conv1b')(input_tensor2)
    x2 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x2)
    
    x2 = Conv2D(128, kernel_size=(3,3), activation='relu', name='conv2b')(x2)
    x2 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x2)
    
    x2 = Conv2D(256, kernel_size=(3,3), activation='relu', name='conv3b')(x2)
    x2 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x2)
    
    x3 = Conv2D(64, kernel_size=(5,5), activation='relu', name='conv1c')(input_tensor3)
    x3 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x3)
    
    x3 = Conv2D(128, kernel_size=(3,3), activation='relu', name='conv2c')(x3)
    x3 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x3)
    
    x3 = Conv2D(256, kernel_size=(3,3), activation='relu', name='conv3c')(x3)
    x3 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x3)
    
    x4 = Conv2D(64, kernel_size=(5,5), activation='relu', name='conv1d')(input_tensor4)
    x4 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x4)
    
    x4 = Conv2D(128, kernel_size=(3,3), activation='relu', name='conv2d')(x4)
    x4 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x4)
    
    x4 = Conv2D(256, kernel_size=(3,3), activation='relu', name='conv3d')(x4)
    x4 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x4)
    
    x = Concatenate([x1, x2, x3, x4])
    
    x = Conv2D(256, kernel_size=(3,3), activation='relu', name='conv4')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None)(x)
    
    output = Conv2D(n_out, kernel_size=(3,3), activation='sigmoid', name='confidence_map')(x)
    
    model = Model([input_tensor1, input_tensor2, input_tensor3, input_tensor4], output)
    
    return model

In [13]:
14*14*2048

401408

In [14]:
401408/(16*16)

1568.0

In [15]:
# warm up model
model = create_model(
    input_shape=(SIZE,SIZE,3), 
    n_out=28)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
bn1 (BatchNormalization)     (None, 512, 512, 3)       12        
_________________________________________________________________
inception_v3 (Model)         (None, 14, 14, 2048)      21802784  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 28)                28700     
Total para

In [16]:
import tensorflow as tf
from tensorflow.python.ops import array_ops

# https://github.com/ailias/Focal-Loss-implement-on-Tensorflow/blob/master/focal_loss.py
def focal_loss_org(prediction_tensor, target_tensor, weights=None, alpha=0.25, gamma=2):
    r"""Compute focal loss for predictions.
        Multi-labels Focal loss formula:
            FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                 ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
    Args:
     prediction_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing the predicted logits for each class
     target_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing one-hot encoded classification targets
     weights: A float tensor of shape [batch_size, num_anchors]
     alpha: A scalar tensor for focal loss alpha hyper-parameter
     gamma: A scalar tensor for focal loss gamma hyper-parameter
    Returns:
        loss: A (scalar) tensor representing the value of the loss function
    """
    sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)


In [17]:
def focal_loss(weights=None, alpha=0.25, gamma=2):
    def focal_loss_my(target_tensor, prediction_tensor, ):
        r"""Compute focal loss for predictions.
            Multi-labels Focal loss formula:
                FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                     ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
        Args:
         prediction_tensor: A float tensor of shape [batch_size, num_anchors,
            num_classes] representing the predicted logits for each class
         target_tensor: A float tensor of shape [batch_size, num_anchors,
            num_classes] representing one-hot encoded classification targets
         weights: A float tensor of shape [batch_size, num_anchors]
         alpha: A scalar tensor for focal loss alpha hyper-parameter
         gamma: A scalar tensor for focal loss gamma hyper-parameter
        Returns:
            loss: A (scalar) tensor representing the value of the loss function
        """
        sigmoid_p = tf.nn.sigmoid(prediction_tensor)
        zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)

        # For poitive prediction, only need consider front part loss, back part is 0;
        # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
        pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)

        # For negative prediction, only need consider back part loss, front part is 0;
        # target_tensor > zeros <=> z=1, so negative coefficient = 0.
        neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
        per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                              - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
        return tf.reduce_sum(per_entry_cross_ent)
#         return K.mean(K.binary_crossentropy(target_tensor, prediction_tensor), axis=-1) + tf.reduce_sum(per_entry_cross_ent)
    return focal_loss_my

In [18]:
def focal_loss_fixed(y_true, y_pred):
    gamma = 2.
    alpha = 0.25
    print(y_pred)
    print(y_true)
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

#     pt_1 = K.clip(pt_1, 1e-3, .999)
#     pt_0 = K.clip(pt_0, 1e-3, .999)

    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))


In [19]:
# def focal_loss(gamma=2., alpha=.25):
#     def focal_loss_fixed(y_true, y_pred):
#         pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
#         pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

#         pt_1 = K.clip(pt_1, 1e-3, .999)
#         pt_0 = K.clip(pt_0, 1e-3, .999)

#         return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
#     return focal_loss_fixed

In [20]:
# create callbacks list
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

epochs = 10; batch_size = 16
checkpoint = ModelCheckpoint('../cache/IV3-75-maximus.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, 
                                   verbose=1, mode='auto', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=6)
callbacks_list = [checkpoint, early, reduceLROnPlat]


In [21]:


# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
np.random.shuffle(indexes)
train_indexes, valid_indexes = train_test_split(indexes, test_size=0.15, random_state=8)

# create train and valid datagens
# train_generator = data_generator.create_train(
#     train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
# validation_generator = data_generator.create_train(
#     train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=True)

In [22]:
# warm up model
model = create_model(
    input_shape=(SIZE,SIZE,3), 
    n_out=28)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
bn1 (BatchNormalization)     (None, 512, 512, 3)       12        
_________________________________________________________________
inception_v3 (Model)         (None, 14, 14, 2048)      21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 28)                28700     
Total para

In [23]:
def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
        print(i, weights[i])
    return weights

In [24]:
y_true = []
y_true_ohe = []
for ii in range(len(train_dataset_info)):
#     if ii != 0:
#         continue
    labels = train_dataset_info[ii]['labels']
#     print(labels)
    y_true.append(labels)
    ohe = np.zeros(28, dtype='int')
    for label in labels:
#         print(label)
        ohe[label] = 1
    y_true_ohe.append(list(ohe))
y_true_ohe = np.array(y_true_ohe)

In [25]:
# y_true_ohe

In [26]:
weights = calculating_class_weights(y_true_ohe)

0 [0.85423654 1.20574311]
1 [ 0.52102757 12.3891547 ]
2 [0.56595388 4.29052748]
3 [0.52644777 9.95259449]
4 [0.53179982 8.36167922]
5 [0.54399664 6.18225229]
6 [ 0.51676424 15.41269841]
7 [0.5499469  5.50531538]
8 [  0.50085432 293.13207547]
9 [  0.50072517 345.24444444]
10 [5.00450973e-01 5.54857143e+02]
11 [ 0.51822943 14.21408966]
12 [ 0.51132175 22.58139535]
13 [ 0.50879319 28.9310987 ]
14 [ 0.51776311 14.57410882]
15 [5.00338153e-01 7.39809524e+02]
16 [ 0.50867658 29.31320755]
17 [ 0.50340224 73.98095238]
18 [ 0.51494862 17.22394678]
19 [ 0.52504224 10.4831309 ]
20 [ 0.50278317 90.3255814 ]
21 [0.5691885  4.11331745]
22 [ 0.51324744 19.37157107]
23 [0.55274487 5.23979764]
24 [ 0.50523577 48.2484472 ]
25 [0.68009105 1.88818668]
26 [ 0.50533437 47.36585366]
27 [5.00177071e-01 1.41236364e+03]


In [27]:


for layer in model.layers:
    layer.trainable = True
model.layers[0].trainable = False
model.layers[1].trainable = False
model.layers[2].trainable = False

# model.layers[-1].trainable = True
# model.layers[-2].trainable = True
# model.layers[-3].trainable = True
# model.layers[-4].trainable = True
# model.layers[-5].trainable = True
# model.layers[-6].trainable = True

In [28]:
# labels = np.zeros((28))
# labels[0] = 1
model.compile(
    loss=[get_weighted_loss(weights)],
    optimizer=Adam(1e-03),
    metrics=[f1])
# model.summary()

In [ ]:
batch_size = 16

# create train and valid datagens
train_generator = data_generator.create_train(
    train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
validation_generator = data_generator.create_train(
    train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=True)

model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=2, 
    verbose=1)

Epoch 1/2
1651/1651 [==============================] - 496s 300ms/step - loss: 1.6240 - f1: 0.0886 - val_loss: 1.5698 - val_f1: 0.0949
Epoch 2/2
1651/1651 [==============================] - 458s 277ms/step - loss: 1.5683 - f1: 0.0938 - val_loss: 1.5701 - val_f1: 0.0968


In [ ]:
# train all layers
epochs=120
for layer in model.layers:
    layer.trainable = True
model.compile(loss=[get_weighted_loss(weights)],
            optimizer=Adam(lr=1e-4),
            metrics=[f1])
# model.load_weights('../cache/IV3-35-maximus.h5')
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=epochs, 
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/120
1651/1651 [==============================] - 944s 572ms/step - loss: 1.5734 - f1: 0.0921 - val_loss: 1.7044 - val_f1: 0.0860

Epoch 00001: val_loss improved from inf to 1.70440, saving model to ../cache/IV3-75-maximus.h5
Epoch 2/120
1651/1651 [==============================] - 916s 555ms/step - loss: 1.5724 - f1: 0.0920 - val_loss: 1.5845 - val_f1: 0.0898

Epoch 00002: val_loss improved from 1.70440 to 1.58446, saving model to ../cache/IV3-75-maximus.h5
Epoch 3/120
1651/1651 [==============================] - 920s 557ms/step - loss: 1.5788 - f1: 0.0920 - val_loss: 1.5810 - val_f1: 0.0958

Epoch 00003: val_loss improved from 1.58446 to 1.58104, saving model to ../cache/IV3-75-maximus.h5
Epoch 4/120
1651/1651 [==============================] - 919s 557ms/step - loss: 1.5598 - f1: 0.0959 - val_loss: 1.5677 - val_f1: 0.0913

Epoch 00004: val_loss improved from 1.58104 to 1.56769, saving model to ../cache/IV3-75-maximus.h5
Epoch 5/120
1651/1651 [==============================] -

In [23]:
# Create submit
submit = pd.read_csv('../data/sample_submission.csv')
predicted = []
draw_predict = []
model = create_model(
    input_shape=(SIZE,SIZE,3), 
    n_out=28)
for layer in model.layers:
    layer.trainable = True
model.compile(loss=f1_loss,
            optimizer=Adam(lr=1e-4),
            metrics=[f1])
model.load_weights('../cache/IV3-37-maximus.h5')
for name in tqdm(submit['Id']):
    path = os.path.join('../data/test/', name)
    image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
    score_predict = model.predict(image[np.newaxis])[0]
    draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict[0]>=0.2]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

submit['Predicted'] = predicted
# np.save('../cache/draw_predict_InceptionV3-30.npy', score_predict)
# submit.to_csv('../submissions/submit_InceptionV3.csv', index=False)

100%|██████████| 11702/11702 [09:08<00:00, 20.96it/s]


In [29]:
submit.to_csv('../submissions/sub35-max.csv', index=False)

In [30]:
#https://stackoverflow.com/questions/1855095/how-to-create-a-zip-archive-of-a-directory
def backup_project_as_zip(project_dir, zip_file):
    assert(os.path.isdir(project_dir))
    assert(os.path.isdir(os.path.dirname(zip_file)))
    shutil.make_archive(zip_file.replace('.zip',''), 'zip', project_dir)
    pass

In [31]:
import datetime, shutil
now = datetime.datetime.now()
print(now)
PROJECT_PATH = '/home/watts/lal/Kaggle/kagglehp/scripts_nbs'
backup_project_as_zip(PROJECT_PATH, '../cache/code.scripts_nbs.%s.zip'%now)

2018-11-26 06:28:07.734445


In [32]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub35-max.csv -m ""

100%|████████████████████████████████████████| 481k/481k [00:14<00:00, 34.5kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationCPU times: user 366 ms, sys: 172 ms, total: 538 ms
Wall time: 17.6 s


In [24]:
predicted = []
for score_predict in tqdm(draw_predict):
    
    label_predict = np.arange(28)[score_predict[0]>=0.5]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub35-max-a.csv', index=False)

100%|██████████| 11702/11702 [00:00<00:00, 101300.98it/s]


In [28]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub35-max-a.csv -m ""
from time import sleep
sleep(20)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 467k/467k [00:13<00:00, 35.6kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName         date                 description  status    publicScore  privateScore  
---------------  -------------------  -----------  --------  -----------  ------------  
sub35-max-a.csv  2018-11-27 09:45:10               complete  0.456        None          
sub36-max.csv    2018-11-27 06:40:12               complete  0.469        None          
sub37-c.csv      2018-11-26 21:04:50               complete  0.437        None          
sub37-b.csv      2018-11-26 21:04:21               complete  0.457        None          
sub37-a.csv      2018-11-26 21:03:26               complete  0.464        None          
sub35-max.csv    2018-11-26 00:58:38               complete  0.473        None          
sub34b-max.csv   2018-11-24 19:03:59               complete  0.459        None          
sub34a-max.csv   2018-11-24 18:50:22               c

In [29]:
predicted = []
for score_predict in tqdm(draw_predict):
    
    label_predict = np.arange(28)[score_predict[0]>=0.45]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub35-max-b.csv', index=False)

100%|██████████| 11702/11702 [00:00<00:00, 81479.84it/s]


In [30]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub35-max-b.csv -m ""
from time import sleep
sleep(20)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 467k/467k [00:12<00:00, 39.1kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName         date                 description  status    publicScore  privateScore  
---------------  -------------------  -----------  --------  -----------  ------------  
sub35-max-b.csv  2018-11-27 09:45:48               complete  0.456        None          
sub35-max-a.csv  2018-11-27 09:45:10               complete  0.456        None          
sub36-max.csv    2018-11-27 06:40:12               complete  0.469        None          
sub37-c.csv      2018-11-26 21:04:50               complete  0.437        None          
sub37-b.csv      2018-11-26 21:04:21               complete  0.457        None          
sub37-a.csv      2018-11-26 21:03:26               complete  0.464        None          
sub35-max.csv    2018-11-26 00:58:38               complete  0.473        None          
sub34b-max.csv   2018-11-24 19:03:59               c

In [31]:
predicted = []
for score_predict in tqdm(draw_predict):
    
    label_predict = np.arange(28)[score_predict[0]>=0.35]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub35-max-c.csv', index=False)

100%|██████████| 11702/11702 [00:00<00:00, 84763.56it/s]


In [32]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub35-max-c.csv -m ""
from time import sleep
sleep(20)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 472k/472k [00:13<00:00, 37.0kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName         date                 description  status    publicScore  privateScore  
---------------  -------------------  -----------  --------  -----------  ------------  
sub35-max-c.csv  2018-11-27 09:47:12               complete  0.458        None          
sub35-max-b.csv  2018-11-27 09:45:48               complete  0.456        None          
sub35-max-a.csv  2018-11-27 09:45:10               complete  0.456        None          
sub36-max.csv    2018-11-27 06:40:12               complete  0.469        None          
sub37-c.csv      2018-11-26 21:04:50               complete  0.437        None          
sub37-b.csv      2018-11-26 21:04:21               complete  0.457        None          
sub37-a.csv      2018-11-26 21:03:26               complete  0.464        None          
sub35-max.csv    2018-11-26 00:58:38               c

In [33]:
predicted = []
for score_predict in tqdm(draw_predict):
    
    label_predict = np.arange(28)[score_predict[0]>=0.25]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub35-max-d.csv', index=False)

100%|██████████| 11702/11702 [00:00<00:00, 83258.69it/s]


In [34]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub35-max-d.csv -m ""
from time import sleep
sleep(20)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 477k/477k [00:13<00:00, 37.3kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName         date                 description  status    publicScore  privateScore  
---------------  -------------------  -----------  --------  -----------  ------------  
sub35-max-d.csv  2018-11-27 09:55:56               complete  0.472        None          
sub35-max-c.csv  2018-11-27 09:47:12               complete  0.458        None          
sub35-max-b.csv  2018-11-27 09:45:48               complete  0.456        None          
sub35-max-a.csv  2018-11-27 09:45:10               complete  0.456        None          
sub36-max.csv    2018-11-27 06:40:12               complete  0.469        None          
sub37-c.csv      2018-11-26 21:04:50               complete  0.437        None          
sub37-b.csv      2018-11-26 21:04:21               complete  0.457        None          
sub37-a.csv      2018-11-26 21:03:26               c

In [33]:
from time import sleep
sleep(60)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName        date                 description  status    publicScore  privateScore  
--------------  -------------------  -----------  --------  -----------  ------------  
sub35-max.csv   2018-11-26 00:58:38               complete  0.473        None          
sub34b-max.csv  2018-11-24 19:03:59               complete  0.459        None          
sub34a-max.csv  2018-11-24 18:50:22               complete  0.469        None          
sub34-max.csv   2018-11-24 17:27:36               complete  0.473        None          
sub33-h.csv     2018-11-24 06:48:41               complete  0.464        None          
sub33-g.csv     2018-11-24 06:46:19               complete  0.472        None          
sub33-c.csv     2018-11-23 11:48:41               complete  0.493        None          
sub33-bb.csv    2018-11-23 11:47:32               complete  0.493        None          
sub33-b.csv     2018-11-23 11:46:26               complete  0.498        None          
sub33-a.csv     2018-11-23 11:45

In [26]:
from time import sleep
sleep(60)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName  date                 description  status    publicScore  privateScore  
--------  -------------------  -----------  --------  -----------  ------------  
sub8.csv  2018-10-20 20:08:45               complete  0.422        None          
sub7.csv  2018-10-20 17:06:09               complete  0.389        None          
sub5.csv  2018-10-19 18:27:33               complete  0.387        None          
sub4.csv  2018-10-19 14:45:15               complete  0.411        None          
sub3.csv  2018-10-19 10:19:26               complete  0.377        None          
sub2.csv  2018-10-19 08:07:30               complete  0.135        None          
sub1.csv  2018-10-19 06:28:57               complete  0.374        None          
